# Analyzing Used Car Listings on eBay Kleinanzeigen

## Import modules and data 

In [71]:
import numpy as np
import pandas as pd
import re

In [72]:
# notice utf-8 decode error, change to use Latin-1
autos = pd.read_csv("autos.csv", encoding="Latin-1")

## Quick overview on data

In [73]:
# print out index dtype, columns, and non-null values inforamtion
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

In [74]:
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


Notice: some columns are missing values, and some columns seem to store numeric values as string.

## Clean columns

In [75]:
# modify the columns name from camelCase to snake_case
new_col = []
for col in autos.columns:
    if col == "powerPS":
        col = "power_ps"
    else:
        col = re.sub(r'(?<!^)(?=[A-Z])', '_', col).lower()
    new_col.append(col)

autos.columns = new_col
print(autos.columns)

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'year_of_registration', 'gearbox', 'power_ps', 'model',
       'odometer', 'month_of_registration', 'fuel_type', 'brand',
       'not_repaired_damage', 'date_created', 'nr_of_pictures', 'postal_code',
       'last_seen'],
      dtype='object')


## Data Exploration and Cleaning

In [76]:
print(autos.describe(include="all"))

               date_crawled         name  seller offer_type  price abtest  \
count                 50000        50000   50000      50000  50000  50000   
unique                48213        38754       2          2   2357      2   
top     2016-04-02 15:49:30  Ford_Fiesta  privat    Angebot     $0   test   
freq                      3           78   49999      49999   1421  25756   
mean                    NaN          NaN     NaN        NaN    NaN    NaN   
std                     NaN          NaN     NaN        NaN    NaN    NaN   
min                     NaN          NaN     NaN        NaN    NaN    NaN   
25%                     NaN          NaN     NaN        NaN    NaN    NaN   
50%                     NaN          NaN     NaN        NaN    NaN    NaN   
75%                     NaN          NaN     NaN        NaN    NaN    NaN   
max                     NaN          NaN     NaN        NaN    NaN    NaN   

       vehicle_type  year_of_registration  gearbox      power_ps  model  \


Notice: 
1. Some columns values seem to be nearly all the same. e.g. seller, offer_type
2. The nr_of_pictures column looks odd (it has only one value nan)

In [77]:
print(autos["nr_of_pictures"].value_counts())
print("\n")
print(autos["seller"].value_counts())
print("\n")
print(autos["offer_type"].value_counts())

0    50000
Name: nr_of_pictures, dtype: int64


privat        49999
gewerblich        1
Name: seller, dtype: int64


Angebot    49999
Gesuch         1
Name: offer_type, dtype: int64


In [78]:
# drop nr_of_pictures column, plus the other two we noted as mostly one value
autos.drop(["nr_of_pictures", "seller", "offer_type"], axis=1).head()

,date_crawled,name,price,abtest,vehicle_type,year_of_registration,gearbox,power_ps,model,odometer,month_of_registration,fuel_type,brand,not_repaired_damage,date_created,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,39218,2016-04-01 14:38:50


In [79]:
# found that the price and odometer columns are numeric values stored as text
autos["price"] = autos["price"].str.replace(",", "")
autos["price"] = autos["price"].str.replace("$", "").astype(int)

autos["odometer"] = autos["odometer"].str.replace(",", "")
autos["odometer"] = autos["odometer"].str.replace("km", "").astype(int)

In [95]:
# rename odometer column to odometer_km
autos.rename({"odometer": "odometer_km"}, axis=1, inplace=True)

## Exploring odometer and price

In [84]:
autos["odometer_km"].value_counts()

150000    32424
125000     5170
100000     2169
90000      1757
80000      1436
70000      1230
60000      1164
50000      1027
5000        967
40000       819
30000       789
20000       784
10000       264
Name: odometer_km, dtype: int64

In [92]:
autos["price"].describe()

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64

In [88]:
autos["price"].value_counts().sort_index(ascending=False).head(20)

99999999    1
27322222    1
12345678    3
11111111    2
10000000    1
3890000     1
1300000     1
1234566     1
999999      2
999990      1
350000      1
345000      1
299000      1
295000      1
265000      1
259000      1
250000      1
220000      1
198000      1
197000      1
Name: price, dtype: int64

In [91]:
autos["price"].value_counts().sort_index(ascending=True).head(20)

0     1421
1      156
2        3
3        1
5        2
8        1
9        1
10       7
11       2
12       3
13       2
14       1
15       2
17       3
18       1
20       4
25       5
29       1
30       7
35       1
Name: price, dtype: int64

Notice: There are a number of listings with prices below 
0. There are also a small number of listings with very high values, including 14 at around or over $1 million.

Given that eBay is an auction site, there could legitimately be items where the opening bid is 1.

Therefore decide to remove anything above $350,000, since it seems that prices begin to jump up to less realistic numbers.

In [93]:
autos = autos[autos["price"].between(1,351000)]
autos["price"].describe()

count     48565.000000
mean       5888.935591
std        9059.854754
min           1.000000
25%        1200.000000
50%        3000.000000
75%        7490.000000
max      350000.000000
Name: price, dtype: float64

## Exploring the date columns

In [99]:
autos[['date_crawled','date_created','last_seen']][0:5]

,date_crawled,date_created,last_seen
0,2016-03-26 17:47:46,2016-03-26 00:00:00,2016-04-06 06:45:54
1,2016-04-04 13:38:56,2016-04-04 00:00:00,2016-04-06 14:45:08
2,2016-03-26 18:57:24,2016-03-26 00:00:00,2016-04-06 20:15:37
3,2016-03-12 16:58:10,2016-03-12 00:00:00,2016-03-15 03:16:28
4,2016-04-01 14:38:50,2016-04-01 00:00:00,2016-04-01 14:38:50


In [112]:
# observe the distribution of values in the date_crawled, ad_created, and last_seen columns
print(autos["date_crawled"].str[:10].value_counts(normalize=True).sort_index())

2016-03-05    0.025327
2016-03-06    0.014043
2016-03-07    0.036014
2016-03-08    0.033296
2016-03-09    0.033090
2016-03-10    0.032184
2016-03-11    0.032575
2016-03-12    0.036920
2016-03-13    0.015670
2016-03-14    0.036549
2016-03-15    0.034284
2016-03-16    0.029610
2016-03-17    0.031628
2016-03-18    0.012911
2016-03-19    0.034778
2016-03-20    0.037887
2016-03-21    0.037373
2016-03-22    0.032987
2016-03-23    0.032225
2016-03-24    0.029342
2016-03-25    0.031607
2016-03-26    0.032204
2016-03-27    0.031092
2016-03-28    0.034860
2016-03-29    0.034099
2016-03-30    0.033687
2016-03-31    0.031834
2016-04-01    0.033687
2016-04-02    0.035478
2016-04-03    0.038608
2016-04-04    0.036487
2016-04-05    0.013096
2016-04-06    0.003171
2016-04-07    0.001400
Name: date_crawled, dtype: float64


In [113]:
print(autos["date_created"].str[:10].value_counts(normalize=True).sort_index())

2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
                ...   
2016-04-03    0.038855
2016-04-04    0.036858
2016-04-05    0.011819
2016-04-06    0.003253
2016-04-07    0.001256
Name: date_created, Length: 76, dtype: float64


In [114]:
print(autos["last_seen"].str[:10].value_counts(normalize=True).sort_index())

2016-03-05    0.001071
2016-03-06    0.004324
2016-03-07    0.005395
2016-03-08    0.007413
2016-03-09    0.009595
2016-03-10    0.010666
2016-03-11    0.012375
2016-03-12    0.023783
2016-03-13    0.008895
2016-03-14    0.012602
2016-03-15    0.015876
2016-03-16    0.016452
2016-03-17    0.028086
2016-03-18    0.007351
2016-03-19    0.015834
2016-03-20    0.020653
2016-03-21    0.020632
2016-03-22    0.021373
2016-03-23    0.018532
2016-03-24    0.019767
2016-03-25    0.019211
2016-03-26    0.016802
2016-03-27    0.015649
2016-03-28    0.020859
2016-03-29    0.022341
2016-03-30    0.024771
2016-03-31    0.023783
2016-04-01    0.022794
2016-04-02    0.024915
2016-04-03    0.025203
2016-04-04    0.024483
2016-04-05    0.124761
2016-04-06    0.221806
2016-04-07    0.131947
Name: last_seen, dtype: float64


In [119]:
# reconcile with the distribution of registration_year
autos["year_of_registration"].describe()

count    46681.000000
mean      2002.910756
std          7.185103
min       1910.000000
25%       1999.000000
50%       2003.000000
75%       2008.000000
max       2016.000000
Name: year_of_registration, dtype: float64

The year_of_registration seems to have odd values. The minimum value is 1000, long before cars were invented and the maximum is 9999, many years into the future.

Because a car can't be first registered before the listing was seen, any vehicle with a registration year above 2016 is definitely inaccurate.

As for the earliest year is more difficult. Realistically, it could be somewhere in the first few decades of the 1900s.

In [118]:
# remove the yeas below 1900 and above 2016
autos = autos[autos["year_of_registration"].between(1900,2016)]

## Exploring Price by Brand

In [122]:
autos["brand"].value_counts(normalize=True)

volkswagen        0.211264
bmw               0.110045
opel              0.107581
mercedes_benz     0.096463
audi              0.086566
ford              0.069900
renault           0.047150
peugeot           0.029841
fiat              0.025642
seat              0.018273
skoda             0.016409
nissan            0.015274
mazda             0.015188
smart             0.014160
citroen           0.014010
toyota            0.012703
hyundai           0.010025
sonstige_autos    0.009811
volvo             0.009147
mini              0.008762
mitsubishi        0.008226
honda             0.007840
kia               0.007069
alfa_romeo        0.006641
porsche           0.006127
suzuki            0.005934
chevrolet         0.005698
chrysler          0.003513
dacia             0.002635
daihatsu          0.002506
jeep              0.002271
subaru            0.002142
land_rover        0.002099
saab              0.001649
jaguar            0.001564
daewoo            0.001500
trabant           0.001392
r

In [128]:
# select brand that have over 5% of the total values
brand = autos["brand"].value_counts(normalize=True)
brand[brand > 0.05].index

Index(['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford'], dtype='object')

German manufacturers represent four out of the top five brands.
Volkswagen is by far the most popular brand, with approximately double the cars for sale of the next two brands combined.

In [171]:
# explore price variations across different car brands
brand_mean_prices = autos.groupby("brand")["price"].mean().drop(brand[brand <= 0.05].index)
print(brand_mean_prices)

brand
audi             9336.687454
bmw              8332.820518
ford             3749.469507
mercedes_benz    8628.450366
opel             2975.241935
volkswagen       5402.410262
Name: price, dtype: float64


Notice:

1. Audi, BMW and Mercedes Benz are more expensive
2. Ford and Opel are less expensive
3. Volkswagen is in between - this may explain its popularity, it may be a 'best of 'both worlds' option

## Exploring Mileage

In [169]:
# explore milage variations across different car brands
brand_mean_milage = round(autos.groupby("brand")["odometer_km"].mean().drop(brand[brand <= 0.05].index))
print(brand_mean_milage)

brand
audi             129157.0
bmw              132573.0
ford             124266.0
mercedes_benz    130788.0
opel             129310.0
volkswagen       128707.0
Name: odometer_km, dtype: float64


In [175]:
# aggregate data from brand_mean_prices, brand_mean_milage to new DataFrame
pd.concat([brand_mean_milage,brand_mean_prices], axis=1)

,odometer_km,price
brand,,
audi,129157.0,9336.687454
bmw,132573.0,8332.820518
ford,124266.0,3749.469507
mercedes_benz,130788.0,8628.450366
opel,129310.0,2975.241935
volkswagen,128707.0,5402.410262


The range of car mileages does not vary as much as the prices do by brand.

There is a slight trend to the more expensive vehicles having higher mileage, with the less expensive vehicles having lower mileage.